<a href="https://colab.research.google.com/github/krcjaro/DS-practicum-2020/blob/master/Lecture05_/Transfer_Learning_DogsAndCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

## Data

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

## MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
________________

## Transfer Learning

### Model

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=2, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Data Generators

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=32
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=32
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Training

In [ ]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

Epoch 1/5
63/63 [==============================] - 17s 275ms/step - loss: 0.2497 - accuracy: 0.8945 - val_loss: 0.1102 - val_accuracy: 0.9570
Epoch 2/5
63/63 [==============================] - 17s 263ms/step - loss: 0.1197 - accuracy: 0.9525 - val_loss: 0.0925 - val_accuracy: 0.9610
Epoch 3/5
63/63 [==============================] - 17s 263ms/step - loss: 0.1110 - accuracy: 0.9605 - val_loss: 0.0862 - val_accuracy: 0.9640
Epoch 4/5
63/63 [==============================] - 17s 262ms/step - loss: 0.1015 - accuracy: 0.9620 - val_loss: 0.0913 - val_accuracy: 0.9650
Epoch 5/5
63/63 [==============================] - 17s 267ms/step - loss: 0.0838 - accuracy: 0.9670 - val_loss: 0.0884 - val_accuracy: 0.9680


## Export to TFjs

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 2.1MB/s 


In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "./export_model")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
import shutil
shutil.make_archive("export_model", 'zip', "export_model/")

from google.colab import files
files.download('export_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>